In [1]:
import numpy as np
import seaborn as sn
from imblearn.over_sampling import SMOTE 
import keras
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from keras import losses
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import class_weight
from sklearn.metrics import recall_score, confusion_matrix, multilabel_confusion_matrix, precision_score, precision_recall_curve, average_precision_score, make_scorer
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from pandas import DataFrame
import xlsxwriter
import time

Training_Data = pd.ExcelFile("Training Data.xlsx") #Training Data already pre-scaled to the IBIS Data set
Label_Data = pd.ExcelFile("Labels.xlsx") #Labels
data = Training_Data.parse(Training_Data.sheet_names[0])
label_data = Label_Data.parse(Label_Data.sheet_names[0])
data_features = data.loc[:, data.columns]
data_features = data_features.drop(['ROI','MATCH DEMOS','INDEX SEX','MATCH BASC2','INDEX GA', 'HYP', 'INDEX MEDU', 'MATCH DEMOS OLD', 'INDEX AGE', 'ATP Label', 'HYP Label'], axis=1)
data_features = data_features.dropna()
data_features = data_features.drop(['ATP'], axis=1)
labels = label_data.loc[:, label_data.columns]
labels = labels.drop(['ROI','MATCH BASC2', 'INDEX ATP', 'INDEX HYP', 'HYP'], axis=1)
labels = labels.dropna()
print(data_features.shape)
print(labels.shape)

seed = 7
print("SEED:",seed)
#Set to the same seed as my gridSearchCV until I can run more gridSearchCVs with random seeds and average them. This is after I try the different class weight gridSearches.
#Once smoted, sex should be rounded to the threshold of 0.5 to make it a binary feature.
#Begin writing thesis

/nas/longleaf/apps/anaconda/2019.10.ood/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(148, 300)
(148, 1)
SEED: 7


In [2]:
data_features = data_features.to_numpy()
labels = labels.to_numpy()

In [3]:
skf = StratifiedKFold(n_splits=8)
print(skf.get_n_splits(data_features, labels))
print(skf)

data_X = pd.DataFrame()
data_Y = pd.DataFrame()
fold=0
for train_index, test_index in skf.split(data_features, labels):
  
    X_train, X_test = data_features[train_index], data_features[test_index]
    Y_train, Y_test = labels[train_index], labels[test_index]
   
    sm = SMOTE(sampling_strategy = 'minority', random_state = seed, k_neighbors=2) 
    X_train_smoted, Y_train_smoted = sm.fit_sample(X_train, Y_train.ravel()) #Only smote the training set.
    
    print("Fold",fold)
    print("X Train Smoted length",len(X_train_smoted),"X Test length", len(X_test))
   
    
    X_train_smoted_df=pd.DataFrame(X_train_smoted)
    data_X = data_X.append(X_train_smoted_df)
    X_test_df=pd.DataFrame(X_test)
    data_X = data_X.append(X_test_df)
    
    Y_train_smoted_df=pd.DataFrame(Y_train_smoted)
    data_Y = data_Y.append(Y_train_smoted_df)
    Y_test_df=pd.DataFrame(Y_test)
    data_Y = data_Y.append(Y_test_df)
    
    fold+=1

8
StratifiedKFold(n_splits=8, random_state=None, shuffle=False)
Fold 0
X Train Smoted length 218 X Test length 19
Fold 1
X Train Smoted length 218 X Test length 19
Fold 2
X Train Smoted length 218 X Test length 19
Fold 3
X Train Smoted length 218 X Test length 19
Fold 4
X Train Smoted length 220 X Test length 18
Fold 5
X Train Smoted length 220 X Test length 18
Fold 6
X Train Smoted length 220 X Test length 18
Fold 7
X Train Smoted length 218 X Test length 18


In [4]:
data_X = data_X.round({296: 0})
data_X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.167723,0.033785,0.149685,0.050587,-0.283762,0.625061,0.187606,-0.197612,0.235310,0.106251,...,0.062580,0.284816,0.407863,0.208103,0.475556,0.086871,0.0,0.750000,14.0,0.991781
1,-0.116216,-0.086548,0.254434,0.164892,-0.229623,0.311852,0.221111,0.103804,0.396944,0.164983,...,0.234383,0.516006,0.501607,0.436699,0.454235,0.317294,1.0,0.750000,9.0,1.016438
2,0.008671,0.016816,0.125914,0.106356,0.018141,0.641574,0.267814,0.074820,0.228598,0.230302,...,0.176464,0.183831,0.357686,0.507466,0.546066,0.457485,0.0,0.666667,12.0,1.071233
3,0.005308,-0.060916,0.193935,0.092845,-0.086244,0.312178,0.034798,-0.244214,0.114453,0.203349,...,0.236034,0.402563,0.560475,0.532426,0.358366,0.435519,0.0,0.916667,15.0,0.978082
4,-0.101819,-0.008209,0.373775,0.071498,-0.014684,0.598363,0.292339,-0.139739,0.062950,-0.127031,...,0.260455,0.132798,0.142365,0.470704,0.228815,0.335065,1.0,0.750000,16.0,1.104110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,0.024301,0.079021,0.257137,0.146070,-0.140987,0.512385,0.029126,0.051665,0.244334,0.182167,...,0.170434,0.504263,0.764572,0.171097,0.276430,0.463951,1.0,0.750000,14.0,1.010959
14,-0.176812,-0.111868,0.145135,0.256893,-0.104476,0.378185,0.233012,-0.234514,0.167317,0.020632,...,0.196369,0.100463,0.061728,0.153907,0.396728,0.404926,0.0,0.750000,18.0,1.013699
15,-0.266167,-0.152861,0.157159,0.057370,-0.143442,0.410800,0.102964,-0.183928,0.202237,0.195613,...,0.188245,0.209554,0.435508,0.113689,0.575967,0.457549,0.0,0.833333,18.0,1.117808
16,0.011006,-0.207553,0.133767,0.080135,-0.076297,0.481662,0.201183,-0.058289,0.326582,-0.093324,...,0.149971,0.437697,0.290071,0.193907,0.479934,0.331472,0.0,0.583333,12.0,1.112329


In [5]:
data_Y

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
13,0.0
14,0.0
15,0.0
16,0.0


In [6]:
print(len(data_X))
print(len(data_Y))

data_Y.to_excel("DATA_Y.xlsx")

data_X.to_excel("DATA_X.xlsx")

1898
1898


In [7]:
string = "["
for x in range(1880,1898):
    string+=str(x)
    string+=","
string +=']'
print(string)

[1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,]


In [8]:
# Function to create model, required for KerasClassifier
def create_model(dropout=0.05, layer1_size=110, layer2_size=20):
    # create model
    model = Sequential()
    model.add(Dense(layer1_size, input_dim=300, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(layer2_size, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def iterator():
    fold0 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217]
    fold0_test = [218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236]
    fold1 = [237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454]
    fold1_test = [455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473]
    fold2 = [474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691]
    fold2_test = [692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710]
    fold3 = [711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928]
    fold3_test = [929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947]
    fold4 = [948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167]
    fold4_test = [1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185]
    fold5 = [1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227,1228,1229,1230,1231,1232,1233,1234,1235,1236,1237,1238,1239,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405]
    fold5_test = [1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423]
    fold6 = [1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643]
    fold6_test = [1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661]
    fold7 = [1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879]
    fold7_test = [1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897]
    
    print("fold0",len(fold0))
    print("fold0_test",len(fold0_test))
    print("fold1",len(fold1))
    print("fold1_test",len(fold1_test))
    print("fold2",len(fold2))
    print("fold2_test",len(fold2_test))
    print("fold3",len(fold3))
    print("fold3_test",len(fold3_test))
    print("fold4",len(fold4))
    print("fold4_test",len(fold4_test))
    print("fold5",len(fold5))
    print("fold5_test",len(fold5_test))
    print("fold6",len(fold6))
    print("fold6_test",len(fold6_test))
    print("fold7",len(fold7))
    print("fold7_test",len(fold7_test))
    
    
    return [(fold0,fold0_test),(fold1,fold1_test),(fold2,fold2_test),(fold3,fold3_test),(fold4,fold4_test),(fold5,fold5_test),(fold6,fold6_test),(fold7,fold7_test)]
custom_cv = iterator()
#Now I have the smoted training sets. How can I:
#A) Duplicate those with atypical labels for class weights
#B) Get their indices into a tuple for the gridsearch

#For the iterable, once you get the training data smoted, add ALL of it with testing in a dataframe. 


#For scoring I can try a weighted accuracy. Or a combination of Recall and PPV.
#Weighted accuracy - (Sensitivty tp/(tp+fn) + Specificity tn/(tn+fp))/2 => This is the same as balanced_accuracy metric in sklearn.
#An iterable yielding (train, test) splits as arrays of indices.
#To work around setting class weights, I can duplicate the minority class points in each fold only on the training data AFTER smote.
#Max amount of compute nodes I can request is 64. Shoot for 16-32 cores. Request more than 4 hours.

#The order for how this works: First I split into train/test/split w/ even amounts of testing in each.
#Then I have to smote the training data. From there I can repeat minority class values? but how
#Have to then return an array containing 2-tuples. The left hand is training indices and the right is testing indices.
#How can I get these indices even after smote?
#From there, I then have to figure out a way to get multiple class weights. Does not solve the duplication I

fold0 218
fold0_test 19
fold1 218
fold1_test 19
fold2 218
fold2_test 19
fold3 218
fold3_test 19
fold4 220
fold4_test 18
fold5 220
fold5_test 18
fold6 220
fold6_test 18
fold7 218
fold7_test 18


In [9]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 15, 20, 25, 30, 35]
epochs = [25, 30, 35, 40, 45, 50, 60, 100, 150]
dropout_rate = [0.1, 0.15, 0.2, 0.25, 0.3]
#class_weights = [{0:1, 1:2},{0:1, 1:2.5},{0:1, 1:3},{0:1, 1:3.5},{0:1, 1:4},{0:1, 1:4.5},{0:1, 1:5}]
#Smote k-nearest neighbors
#Hidden layer sizes
hidden_size1 = [100, 110, 120, 130]
hidden_size2 = [15, 20, 25, 30, 35]

param_grid = dict(batch_size=batch_size, epochs=epochs, dropout=dropout_rate, layer1_size=hidden_size1, layer2_size=hidden_size2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='balanced_accuracy', n_jobs=-1, cv=custom_cv)
grid_result = grid.fit(data_X,data_Y)


#For scoring I can try a weighted accuracy. Or a combination of Recall and PPV.
#Weighted accuracy - (Sensitivty tp/(tp+fn) + Specificity tn/(tn+fp))/2 => This is true
#An iterable yielding (train, test) splits as arrays of indices.
#To work around setting class weights, I can duplicate the minority class points in each fold only on the training data AFTER smote.
#Max amount of compute nodes I can request is 64. Shoot for 16-32 cores. Request more than 4 hours.

/nas/longleaf/apps/anaconda/2019.10.ood/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [10]:
df = pd.DataFrame(grid_result.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_dropout,param_epochs,param_layer1_size,param_layer2_size,params,...,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score
0,1.330759,0.333169,0.190484,0.042640,10,0.1,25,100,15,"{'batch_size': 10, 'dropout': 0.1, 'epochs': 2...",...,0.5,0.406250,0.770833,0.400000,0.466667,0.5,0.75000,0.553646,0.137416,4179
1,1.010571,0.133364,0.188895,0.051867,10,0.1,25,100,20,"{'batch_size': 10, 'dropout': 0.1, 'epochs': 2...",...,0.5,0.437500,0.635417,0.433333,0.633333,0.5,0.75000,0.565625,0.106566,3170
2,1.016160,0.167274,0.176415,0.056135,10,0.1,25,100,25,"{'batch_size': 10, 'dropout': 0.1, 'epochs': 2...",...,0.5,0.375000,0.604167,0.466667,0.466667,0.5,0.75000,0.533333,0.107912,5052
3,1.029211,0.162536,0.197856,0.066430,10,0.1,25,100,30,"{'batch_size': 10, 'dropout': 0.1, 'epochs': 2...",...,0.5,0.406250,0.802083,0.466667,0.466667,0.5,0.75000,0.565885,0.136187,2838
4,1.191386,0.271567,0.160865,0.060197,10,0.1,25,100,35,"{'batch_size': 10, 'dropout': 0.1, 'epochs': 2...",...,0.5,0.572917,0.770833,0.466667,0.633333,0.5,0.75000,0.603646,0.107503,526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,1.909881,0.271689,0.275213,0.139275,35,0.3,150,130,15,"{'batch_size': 35, 'dropout': 0.3, 'epochs': 1...",...,0.5,0.406250,0.635417,0.466667,0.800000,0.5,0.71875,0.582812,0.127261,1632
5396,1.838767,0.270191,0.155825,0.045896,35,0.3,150,130,20,"{'batch_size': 35, 'dropout': 0.3, 'epochs': 1...",...,0.5,0.468750,0.635417,0.466667,0.633333,0.5,0.75000,0.573698,0.097205,2536
5397,1.917086,0.350672,0.175790,0.053200,35,0.3,150,130,25,"{'batch_size': 35, 'dropout': 0.3, 'epochs': 1...",...,0.5,0.406250,0.635417,0.466667,0.633333,0.5,0.75000,0.565885,0.107312,2838
5398,1.913349,0.363699,0.195256,0.093150,35,0.3,150,130,30,"{'batch_size': 35, 'dropout': 0.3, 'epochs': 1...",...,0.5,0.406250,0.635417,0.466667,0.633333,0.5,0.71875,0.561979,0.100917,3215


In [11]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.661979 using {'batch_size': 30, 'dropout': 0.15, 'epochs': 25, 'layer1_size': 110, 'layer2_size': 35}
0.553646 (0.137416) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 100, 'layer2_size': 15}
0.565625 (0.106566) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 100, 'layer2_size': 20}
0.533333 (0.107912) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 100, 'layer2_size': 25}
0.565885 (0.136187) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 100, 'layer2_size': 30}
0.603646 (0.107503) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 100, 'layer2_size': 35}
0.570312 (0.125324) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 110, 'layer2_size': 15}
0.586719 (0.131774) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 110, 'layer2_size': 20}
0.603646 (0.135900) with: {'batch_size': 10, 'dropout': 0.1, 'epochs': 25, 'layer1_size': 11